# BINGO Hackaton - Lecture 05

**Luciano Barosi**

*BINGO Collaboration*

## Fast Fourier Transform and Applications

In [ ]:
# Make plots appear inline, set custom plotting style
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import fftpack

In [ ]:
# show result from all calculations of the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Building an harmonic signal
f = 10  # Frequency, in cycles per second, or Hertz
f_s = 100  # Sampling rate, or number of measurements per second

t = np.linspace(0, 2, 2 * f_s, endpoint=False)
x = np.sin(f * 2 * np.pi * t)

fig, ax = plt.subplots()
ax.plot(t, x)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Signal amplitude');

In [ ]:
from scipy import fftpack

X = fftpack.fft(x)
freqs = fftpack.fftfreq(len(x)) * f_s

fig, ax = plt.subplots()

ax.stem(freqs, np.abs(X), use_line_collection = True)
ax.set_xlabel('Frequency in Hertz [Hz]')
ax.set_ylabel('Frequency Domain (Spectrum) Magnitude')
ax.set_xlim(-f_s / 2, f_s / 2)
ax.set_ylim(-5, 110)

We want to study signals and decompose them in their constituent frequencies.

In [ ]:
from IPython.display import Audio
file = "../DATA/Nightingale-sound.wav"
Audio(file)

In [ ]:
from scipy.io import wavfile

rate, audio = wavfile.read(file)
print(rate)


In [ ]:
N = audio.shape[0]
L = N / rate

print(f'Audio length: {L:.2f} seconds')

f, ax = plt.subplots(figsize=(7.2, 2.4))
ax.plot(np.arange(N) / rate, audio)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Amplitude [unknown]');

In order to understand better the signal we are going to construct a moving window of length 1024 samples, each one 100 samples apart. It is important that the size of the windows is a multiple of 2.

The edges of the window may introduce artifacts in the signal. We are going to use a smooth window.

In [ ]:
# Code directly obtained from https://docs.scipy.org/doc/numpy/reference/generated/numpy.hanning.html
size_w = 50 #size of the window
size_s = 2048 # size of the sample
window = np.hanning(size_w)

A = fftpack.fft(window, size_s) / (size_w/2)
# Center frequencies
mag = np.abs(fftpack.fftshift(A))
freq = np.linspace(-0.5, 0.5, len(A))
# Proper care with logs
with np.errstate(divide='ignore', invalid='ignore'):
    response = 20 * np.log10(mag)
# Clipping signal frequencies
#response = np.clip(response, -100, 100)

fig, ax = plt.subplots(ncols = 2, figsize = (18,6))
ax[0].plot(window)
ax[0].set_title("Hann window")
ax[0].set_ylabel("Amplitude")
ax[0].set_xlabel("Sample")

ax[1].plot(freq, response)
ax[1].set_title("Frequency response of the Hann window")
ax[1].set_ylabel("Magnitude [dB]")
ax[1].set_xlabel("Normalized frequency [cycles per sample]")

plt.show();

In [ ]:
from skimage import util
M = 1024

slices = util.view_as_windows(audio, window_shape = (M,), step = 100)
print(f'Audio shape: {audio.shape}, Sliced audio shape: {slices.shape}')

In [ ]:
# Construct a Hanning window
win = np.hanning(M + 1)[:-1]
slices = slices * win
# Transpose
slices = slices.T
print('Shape of `slices`:', slices.shape)
# DFT for each slice
spectrum = np.fft.fft(slices, axis=0)[:M // 2 + 1:-1]
spectrum = np.abs(spectrum)


In [ ]:
f, ax = plt.subplots(figsize=(7.2, 2.4))

S = np.abs(spectrum)
S = 20 * np.log10(S / np.max(S))

ax.imshow(S, origin='lower', cmap='viridis',
          extent=(0, L, 0, rate / 2 / 1000))
ax.axis('tight')
ax.set_ylabel('Frequency [kHz]')
ax.set_xlabel('Time [s]');

In [ ]:
# There is a package for that. This is a spectogram.
from scipy import signal

freqs, times, Sx = signal.spectrogram(audio, fs=rate, window='hanning',
                                      nperseg=4*1024, noverlap=M - 1000,
                                      detrend=False, scaling='spectrum')

f, ax = plt.subplots(figsize=(16, 6))
# pcolormesh is an useful function: given a tuple (x, y, z) it plots a density map.
ax.pcolormesh(times, freqs / 1000, 10 * np.log10(Sx), cmap='viridis')
ax.set_ylabel('Frequency [kHz]')
ax.set_xlabel('Time [s]')
plt.show();

### Noisy Signal

In [ ]:
samp_freq = 50                             # Sampling frequency
time_step = 1/samp_freq                    # Sampling period
N = 1024                                    # Total number of samples
n = np.arange(0,N-1)                       # Samples
time_vec = time_step*(n)                          # Sampled times

period = 2
seed = 1000
signal = np.cos(2*np.pi*time_vec/period) + 0.1 * np.random.randn(time_vec.size)  # Noisy signal


In [ ]:
# Plot the signal
plt.figure(figsize=(16, 6))
plt.plot(time_vec, signal, label='Original signal')
plt.show();

In [ ]:
# The FFT of the signal
sig_fft = fftpack.fft(signal)

# And the power (sig_fft is of complex dtype)
power = np.abs(sig_fft)

# The corresponding frequencies
sample_freq = fftpack.fftfreq(signal.size, d=time_step)

# Plot the FFT power
plt.figure(figsize=(10, 5))
plt.plot(sample_freq, power)
plt.xlabel('Frequency [Hz]')
plt.ylabel('plower')

# Find the peak frequency: we can focus on only the positive frequencies
pos_mask = np.where(sample_freq > 0)
freqs = sample_freq[pos_mask]
peak_freq = freqs[power[pos_mask].argmax()]

# An inner plot to show the peak frequency
axes = plt.axes([0.55, 0.3, 0.3, 0.5])
plt.title('Peak frequency')
plt.plot(sample_freq, power)
plt.xlim(0.8*peak_freq, 1.2*peak_freq)
plt.setp(axes, yticks=[])

# Check that it does indeed correspond to the frequency that we generate
# the signal with
print(f"{peak_freq:.2f}, {(1/period):.2f} ")
plt.show();


In [ ]:
# Filtering high frequencies brutally
high_freq_fft = sig_fft.copy()
high_freq_fft[np.abs(sample_freq) > peak_freq] = 0
filtered_sig = fftpack.ifft(high_freq_fft)

plt.figure(figsize=(10, 5))
plt.plot(time_vec, signal, label='Original signal')
plt.plot(time_vec, filtered_sig, linewidth=3, label='Filtered signal')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.legend(loc='best')
plt.show();


### A few concepts

#### Power Spectral Density

##### Fourier Transform

$$ H(f) = \int_{-\infty}^{\infty} \; h(t) \; e^{-2\pi i f t}dt $$
$$ h(f) = \int_{-\infty}^{\infty} \; H(f) \; e^{-2\pi i f t}df $$

$$\mathrm{PSD}(f) \equiv \| H(f) \|^2 + \| H(-f) \|^2 $$

the **power spectral density** defined above measures the amount of power contained in the frequency interval between $f$ and $f+df$.

#### Convolution

$$ (a \star b)(t) \equiv \int_{-\infty}^\infty \; a(t') b(t-t') dt' $$
$$ h = a\star b \Rightarrow H(f) = A(f) B(f) $$ 

#### Nyquist Sampling Theorem and Aliasing

For discrete time we have a **discrete fourier transform**, and since this is a very time consuming operation, the algorith for calculating became known, which is the **fast fourier transform**.

$$H_k = \sum_{j=0}^{N-1} h_j e^{-\frac{2 \pi i j k}{N}}$$
$$h_j = \frac{1}{N} \sum_{j=0}^{N-1} H_k e^{-\frac{2 \pi i j k}{N}}$$

- A function h[t] of a discrete time variable is called **band limited** if  its Fourier transform is zero for frequencies higher than a certain maximum frequency $|H(f) = 0|$ if $|f|>f_c$.  This frequency is called **Nyquist critical frequency**. 
- **Nyquist theorem** states that, if h(t) is band limited with critical frequency $f_c$, there is a resolution limit in t space, $t_c = 1/(2 f_c$ below which h(t) appears smooth and h(t) can be exactly reconstructed from evenly sampled data.
- The reconstruction is done via the form known as **Whittaker-Shannon** sinc-shifting formula.
- The discrete analog of Power Spectral Density is:
$$ \mathrm{PSD}(f_k) = (\Delta t)^2 (|H_k|^2 + |H_{N-k}|^2) $$

**Reference:**
1. Elegant Scipy (https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch04.html)
2. https://github.com/AllenDowney/ThinkDSP/tree/master/code
3. https://longbaonguyen.github.io/courses/dft/discrete_fourier.html

**Note:** Most of the material follows closely the first reference.

## Aliasing

In [ ]:
# Code based in https://gist.github.com/stephenHartzell/dsp
# Improvement in visualization
samp_freq = 10                             # Sampling frequency
samp_perd = 1/samp_freq                    # Sampling period
N = 32                                     # Total number of samples
n = np.arange(0,N-1)                       # Samples
t = samp_perd*(n)                          # Sampled times
t_fine = np.arange(0,np.max(t), 0.01)      # Finely sampled times
y = np.cos(2*np.pi*t)                      # Discrete 1 Hz signal
yp = np.cos(18*np.pi*t)                    # Discrete 9 Hz signal


fig, ax = plt.subplots(ncols = 2, figsize = (16,6))
# Plot the 1 Hz signal and its discrete counter-part
markerline, stemlines, baseline  = ax[0].stem(t, y, use_line_collection=True, linefmt='red')
markerline.set_markerfacecolor('violet')
ax[0].plot(t_fine, np.cos(2*np.pi*t_fine), alpha = 0.5, linestyle = "dashed")   
ax[0].set_title('Sampled 1 Hz Signal with Fs = {}'.format(samp_freq))
# Plot the 9 Hz signal and its discrete counter-part
markerline, stemlines, baseline  = ax[1].stem(t, yp, use_line_collection=True, linefmt='red')
ax[1].plot(t_fine,np.cos(18*np.pi*t_fine))
markerline.set_markerfacecolor('violet')
ax[1].set_title('Sampled 9 Hz Signal with Fs = {}'.format(samp_freq))
plt.show();

### Fitting Periodic Data
code from Jake VanderPlas, in http://astroML.github.com

In [ ]:
from astroML.datasets import fetch_rrlyrae_templates

In [ ]:
templates =  fetch_rrlyrae_templates()
x, y = templates["115r"].T

In [ ]:
# Pick up two the 3 frequencies to reconstruct the signal
k = 3
y_fft = fftpack.fft(y)
# zero higher frequencies
y_fft[k+1:] = 0 
# get the inverse
y_fit = fftpack.ifft(y_fft).real

In [ ]:
fig = plt.figure()
# plotting two periods each. That is what concatenate does
plt.plot(np.concatenate([x, 1 + x]),
            np.concatenate([y, y]), '--k', lw=2)
plt.plot(np.concatenate([x, 1 + x]),
            np.concatenate([y_fit, y_fit]), color='gray')
plt.show()